In [ ]:
import pandas as pd
import os
from IPython.display import display

date_list = ['2019-20', '2020-21', '2021-22', '2022-23', '2023-24']
player_names = ['Jayson Tatum', 'Devin Booker']
player_dataframes = {}  # Dictionary to store each player's DataFrame

for player in player_names:
    player_combined_df = pd.DataFrame()  # Initialize a DataFrame for the current player
    
    for date in date_list:
        path = fr"C:\Users\mandy\Documents\Github\web_scraping_sports\nba_ph_csv\season_{date}\all_quarters\{player}_content.csv"
        
        if os.path.exists(path):
            season_df = pd.read_csv(path)
            print(f"Found file: {path}")

            season_df['season'] = date  # Add the season column
            player_combined_df = pd.concat([player_combined_df, season_df], ignore_index=True)
        else:
            print(f"{player} {date} not found")
    
    # Add the player's DataFrame to the dictionary
    player_dataframes[player] = player_combined_df

# Example: Access each player's DataFrame
for player, df in player_dataframes.items():
    print(f"Data for {player}:")
    display(df)  # Display the first few rows of each player's DataFrame


In [ ]:
import pandas as pd
import os

date_list = ['2019-20', '2020-21', '2021-22', '2022-23', '2023-24']
player_name = ['Jayson Tatum', 'Devin Booker']
player_dataframes = {}  # Dictionary to store each player's DataFrame

for player in player_name:
    single_player_df = pd.DataFrame()  # Initialize a DataFrame for the current player
    defense_df = pd.DataFrame()  # Initialize a DataFrame for the current player's defense
    
    for date in date_list:
        path = fr"C:\Users\mandy\Documents\Github\web_scraping_sports\historic_player_data\nba_ph_csv\season_{date}\all_quarters\{player}_content.csv"
        path_defense = fr'C:\Users\mandy\Documents\Github\web_scraping_sports\historic_defense_data\nba_dh_csv\defense_csv_{date}\all_quarter_defense_content.csv'

        if os.path.exists(path) and os.path.exists(path_defense):
            #player data
            season_df_player = pd.read_csv(path)
            season_df_player['season'] = date
            single_player_df = pd.concat([single_player_df, season_df_player], ignore_index=True)
            #defense data
            defense_df_season = pd.read_csv(path_defense)
            defense_df_season['season'] = date
            defense_df = pd.concat([defense_df, defense_df_season], ignore_index=True)
            
        else:
            print(f'{player} {date} not found')

    # Add the player's DataFrame to the dictionary
    player_dataframes[player] = single_player_df
    #print(defense_df[defense_df['season'] < '2022-23'])
#player_dataframes['Devin Booker'].head(20)
defense_df

#     jayson_df = player_dataframes['Jayson Tatum']
# jayson_df.head(20)

# # for player, df in player_dataframes.items():
# #     print(f"Data for {player}:")
    
#     for df in player_dataframes.values():
#         print(df.head())
#      # Display the first few rows of each player's DataFrame


In [13]:
import pandas as pd
import os
from IPython.display import display
import numpy as np
from pathlib import Path



date_list = ['2019-20', '2020-21', '2021-22', '2022-23', '2023-24']
player_name = ['Shai Gilgeous-Alexander', 'Cason Wallace', 'Alex Caruso', 'Jalen Williams', 'Isaiah Hartenstein']
player_dataframes = {}  # Dictionary to store each player's DataFrame

for player in player_name:
    single_player_df = pd.DataFrame()  # Initialize a DataFrame for the current player
    defense_df = pd.DataFrame()  # Initialize a DataFrame for the current player's defense
    
    for date in date_list:
        notebook_dir = Path().resolve()
        data_path = notebook_dir / f"../../historic_player_data/nba_ph_csv/season_{date}/all_quarters/{player}_content.csv"
        data_path_defense = notebook_dir / f"../../historic_defense_data/nba_dh_csv/defense_csv_{date}/all_quarter_defense_content.csv"

        path = data_path.resolve()
        path_defense = data_path_defense.resolve()
        if os.path.exists(path) and os.path.exists(path_defense):
            #player data
            season_df_player = pd.read_csv(path)
            season_df_player['season'] = date
            single_player_df = pd.concat([single_player_df, season_df_player], ignore_index=True)
            #defense data
            defense_df_season = pd.read_csv(path_defense)
            defense_df_season['season_defense'] = date
            defense_df = pd.concat([defense_df, defense_df_season], ignore_index=True)
            
        else:
            print(f'{date} not found either for {player} or defense')
    
    # Add the player's DataFrame to the dictionary 
    pd.set_option('display.max_rows', 1000)  # Maximum number of rows to display
    pd.set_option('display.max_columns', None)  # Show all columns
    pd.set_option('display.width', 1000)  # Adjust column width for better readability
    
    player_dataframes[player] = single_player_df

    df_defense = defense_df[['TEAM', 'DefRtg', 'season_defense']]

    merged_df = pd.merge(player_dataframes[player], df_defense, how='inner', left_on=['Away','season'], right_on=['TEAM','season_defense']).reset_index(drop=True)
    merged_df = merged_df.sort_values(by="Date")

    train_data = merged_df[merged_df['Date'] < '2023-01-01']  # Training: First 4 seasons
    test_data = merged_df[merged_df['Date'] >= '2023-01-01']  # Testing: Most recent season

    # Select features and target
    what_train = train_data[['MIN', 'FGA', 'FG%', 'FTA', 'FGM', 'FTM', 'DefRtg']]
    what_test = test_data[['MIN', 'FGA', 'FG%', 'FTA', 'FGM', 'FTM', 'DefRtg']]

    # X_train = train_data[['MIN', 'FGA', 'FG%', 'FTA', 'DefRtg']]
    X_train = what_train
    y_train = train_data['PTS']
    # X_test = test_data[['MIN', 'FGA', 'FG%', 'FTA', 'DefRtg']]
    X_test = what_test
    y_test = test_data['PTS']

    try:
        # Train model
        from sklearn.linear_model import LinearRegression
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Test model
        predictions = model.predict(X_test)   

        # Evaluate
        from sklearn.metrics import mean_absolute_error, mean_squared_error
        mae = mean_absolute_error(y_test, predictions)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))

        # Compare predictions with actual points
        predicted_vs_actual = pd.DataFrame({
        "Date": test_data['Date'],  
        "Matchup": test_data['Matchup'],  # or adjust based on how teams are stored
        "Predicted Points": predictions.round(1), # Adjust for underestimation we want underestimation,
        "Actual Points": y_test}).reset_index(drop=True)

        display(predicted_vs_actual.head(5))

        print(f"{player} MAE: {mae:.2f}, RMSE: {rmse:.2f}")

        # Combine train and test data for final training
        X_full = pd.concat([X_train, X_test], ignore_index=True)
        y_full = pd.concat([y_train, y_test], ignore_index=True)

        # Train the model on all historical data
        model.fit(X_full, y_full)

        recent_games = merged_df[-5:]  # Last 5 games
        avg_min = recent_games['MIN'].mean()
        avg_fga = recent_games['FGA'].mean()
        avg_fg = recent_games['FG%'].mean()
        avg_fta = recent_games['FTA'].mean()


        X_future = pd.DataFrame({
        'MIN': avg_min,         # Projected minutes played
        'FGA': [14.76],         # Projected field goal attempts
        'FG%': [58.631],         # Projected field goal percentage
        'FTA': [5.73],          # Projected free throw attempts
        'DefRtg': [116.3]     # Opponent defensive rating
    })
    
    except Exception as e:
        print(f"{e}")

    # predicted_points = model.predict(X_future)
    # predicted_points = predicted_points.round(1) - 5
    # print(f"{player} Predicted Points: {predicted_points[0]:.1f}")

    # display(X_future)




,Date,Matchup,Predicted Points,Actual Points
0,2023-01-04,OKC @ ORL,33.9,33
1,2023-01-06,OKC vs. WAS,30.7,30
2,2023-01-08,OKC vs. DAL,34.5,33
3,2023-01-10,OKC @ MIA,25.7,26
4,2023-01-12,OKC @ PHI,37.1,37


Shai Gilgeous-Alexander MAE: 0.77, RMSE: 1.01
2019-20 not found either for Cason Wallace or defense
2020-21 not found either for Cason Wallace or defense
2021-22 not found either for Cason Wallace or defense
2022-23 not found either for Cason Wallace or defense
Found array with 0 sample(s) (shape=(0, 7)) while a minimum of 1 is required by LinearRegression.


,Date,Matchup,Predicted Points,Actual Points
0,2023-01-02,CHI @ CLE,4.7,4
1,2023-01-04,CHI vs. BKN,2.4,2
2,2023-01-09,CHI @ BOS,4.8,6
3,2023-01-11,CHI @ WAS,7.1,6
4,2023-01-13,CHI vs. OKC,10.6,11


Alex Caruso MAE: 0.76, RMSE: 1.05
2019-20 not found either for Jalen Williams or defense
2020-21 not found either for Jalen Williams or defense
2021-22 not found either for Jalen Williams or defense


,Date,Matchup,Predicted Points,Actual Points
0,2023-01-03,OKC vs. BOS,24.9,21
1,2023-01-04,OKC @ ORL,5.6,4
2,2023-01-06,OKC vs. WAS,17.4,18
3,2023-01-08,OKC vs. DAL,9.5,10
4,2023-01-10,OKC @ MIA,20.7,12


Jalen Williams MAE: 4.26, RMSE: 5.41


,Date,Matchup,Predicted Points,Actual Points
0,2023-01-02,NYK vs. PHX,3.1,3
1,2023-01-04,NYK vs. SAS,8.2,8
2,2023-01-06,NYK @ TOR,2.2,2
3,2023-01-09,NYK vs. MIL,0.3,0
4,2023-01-11,NYK vs. IND,-0.1,0


Isaiah Hartenstein MAE: 0.26, RMSE: 0.30
